<a href="https://colab.research.google.com/github/U-CovUni/NLP-CourseWork1/blob/main/Stance_Detection_Using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

# Loading the dataset
file_path = '/content/drive/MyDrive/SemEval_in_tweets/semeval.csv'
data = pd.read_csv(file_path)

# Converting the stance labels to numerical format
stance_mapping = {'AGAINST': 0, 'FAVOR': 1, 'NONE': 2}
data['Stance'] = data['Stance'].map(stance_mapping)

# Splitting the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Initializing tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Converting the data into InputExamples
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
    train_InputExamples = train.apply(lambda x: InputExample(guid=None,
                                                             text_a = x[DATA_COLUMN],
                                                             text_b = None,
                                                             label = x[LABEL_COLUMN]), axis = 1)

    validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                                 text_a = x[DATA_COLUMN],
                                                                 text_b = None,
                                                                 label = x[LABEL_COLUMN]), axis = 1)

    return train_InputExamples, validation_InputExamples

train_InputExamples, validation_InputExamples = convert_data_to_examples(train_data,
                                                                         test_data,
                                                                         'Tweet',
                                                                         'Stance')

# Converting InputExamples to InputFeatures
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
                                                     input_dict["token_type_ids"],
                                                     input_dict["attention_mask"])

        features.append(
            InputFeatures(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          label=e.label)
        )

    def gen():
        for f in features:
            yield ({'input_ids': f.input_ids,
                    'attention_mask': f.attention_mask,
                    'token_type_ids': f.token_type_ids}, f.label)

    return tf.data.Dataset.from_generator(
        gen,
        ({'input_ids': tf.int32,
          'attention_mask': tf.int32,
          'token_type_ids': tf.int32},
         tf.int64),
        ({'input_ids': tf.TensorShape([None]),
          'attention_mask': tf.TensorShape([None]),
          'token_type_ids': tf.TensorShape([None])},
         tf.TensorShape([]))
    )

train_dataset = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)

validation_dataset = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_dataset = validation_dataset.batch(32)

# Loading the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Training the model
model.fit(train_dataset, epochs=3, validation_data=validation_dataset)

# Evaluating the model
loss, accuracy = model.evaluate(validation_dataset)
print(f'Test Accuracy: {accuracy}')

# Saving the model
model.save_pretrained('bert_stance_detection_model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
142/142 [==============================] - 6521s 45s/step - loss: 0.8545 - sparse_categorical_accuracy: 0.6022 - val_loss: 0.7781 - val_sparse_categorical_accuracy: 0.6661
Epoch 2/3
142/142 [==============================] - 6392s 45s/step - loss: 0.3595 - sparse_categorical_accuracy: 0.8687 - val_loss: 0.9540 - val_sparse_categorical_accuracy: 0.6980
Epoch 3/3
18/18 [==============================] - 236s 13s/step - loss: 1.2202 - sparse_categorical_accuracy: 0.6980
Test Accuracy: 0.6980462074279785
